In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import xrft as xf

In [ ]:
# !pip install xrft

In [ ]:
theta1 = 0.005
theta = str(theta1)
N21 = 1e-5
N2 = "9.9e-6"
Vinf1 = 0.05
Vinf = str(Vinf1)
h = str(104.0)
f = 10**(-4)
gm = (1+theta1**2*N21*f**(-2))**(-1)
beta = (1+theta1**2*N21*f**(-2))**(0.5)
fstar = (1e-8+theta1**2*N21)**(0.5)
lmbd = N21*theta1*gm/f
delta = 0.1
bo = 0

In [ ]:
# file = "olddata/dataflow_fields_height_103.0_theta_0.005_stratification_1.0e-5_interior_velocity_0.05.nc"
file = "/new_data/paper_data/flow_fields_height_250.00000000000003_theta_0.05_stratification_1.0e-6_interior_velocity_0.1_delta_0.5_bo_0_visc_5.0e-5.nc"
ocean_data=xr.open_dataset("/glade/derecho/scratch/knudsenl/data/"+file)#.isel(time=slice(0,4000)) #i=103
seconds = ocean_data.time/pd.Timedelta("1s")
ocean_data = ocean_data.assign_coords(time = ('time',seconds.data))

In [ ]:
ocean_data

In [ ]:
ocean_data.time[-1]*f/(2*np.pi)

In [ ]:
import matplotlib.colors as colors

In [ ]:
(ocean_data.v).isel(time=-1).plot()


In [ ]:
(ocean_data.ba).isel(time=0).plot()

In [ ]:
inertial_period = ocean_data.time*fstar/(2*np.pi)
ocean_data_ip = ocean_data.assign_coords(intp = ('time',inertial_period.data))

In [ ]:
tau = 30
ocean_data_ip.v.isel(time=tau).plot()
print(ocean_data_ip.intp.isel(time=tau))

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(60, 200)), ocean_data_ip.v.isel(zC=79, yC=0, xC=128, time=slice(60, 200))/ocean_data_ip.v.isel(zC=79, yC=0, xC=128, time=60))
# plt.plot(ocean_data_ip.intp.isel(time=slice(100,400)),ocean_data_ip.u.isel(zC=63,yC=0,xC=128,time=slice(100,400))/ocean_data_ip.v.isel(zC=63,yC=0,xC=128,time=100))
# plt.plot(ocean_data_ip.intp.isel(time=slice(100,400)),ocean_data_ip.w.isel(zC=63,yC=0,xC=128,time=slice(100,400))/ocean_data_ip.v.isel(zC=63,yC=0,xC=128,time=100))
# plt.yscale("symlog",linthresh=0.01)

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(100, 420)), ocean_data_ip.v.isel(zC=63, yC=0, xC=128, time=slice(100, 420)))
# plt.plot(ocean_data_ip.intp.isel(time=slice(100,400)),ocean_data_ip.u.isel(zC=63,yC=0,xC=128,time=slice(100,400))/ocean_data_ip.v.isel(zC=63,yC=0,xC=128,time=100))
# plt.plot(ocean_data_ip.intp.isel(time=slice(100,400)),ocean_data_ip.w.isel(zC=63,yC=0,xC=128,time=slice(100,400))/ocean_data_ip.v.isel(zC=63,yC=0,xC=128,time=100))
# plt.yscale("symlog",linthresh=0.01)

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(200,400)),ocean_data_ip.va.isel(zC=63,yF=0,xC=256,time=slice(200,400)))
plt.yscale("symlog",linthresh=0.0001)

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(200,400)),ocean_data_ip.vb.isel(zC=63,yC=0,xF=256,time=slice(200,400)))
plt.yscale("symlog",linthresh=0.0001)

In [ ]:
total = (ocean_data_ip.vb.interp_like(ocean_data_ip.va.xC).interp_like(ocean_data_ip.va.yF)).isel(zC=63,yC=0,xF=256)+ ocean_data_ip.va.isel(zC=63,yF=0,xC=256)

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(200,400)),total.isel(time=slice(200,400)))
plt.yscale("symlog",linthresh=0.0001)

In [ ]:
zi = 155
ti = 100
tf =  -1
# v_mean = ocean_data.v.mean(["xC"])
fourier_data = ocean_data.v.isel(yC=0,zC=zi,xC=256,time=slice(ti,tf))/ocean_data.v.isel(yC=0,zC=zi, xC=256,time=ti).max()
time_data = ocean_data_ip.intp.isel(time=slice(ti,tf))

In [ ]:
# fourier_data["time"] = fourier_data.time * f/(2*np.pi)
plt.plot(time_data,fourier_data)

In [ ]:
# fourier_data_ni = fourier_data.drop(['intp'])
transform = xf.fft(fourier_data,dim="time")

In [ ]:
plt.plot(transform.freq_time*2*np.pi,np.abs(transform))
plt.vlines(0.5*beta*f,ymin=0,ymax=np.max(np.abs(transform))*1.1,color="b",alpha=0.5,linestyles="dotted")
plt.vlines(1.5*beta*f,ymin=0,ymax=np.max(np.abs(transform))*1.1,color="b",alpha=0.5,linestyles="dotted")

In [ ]:
for j in [*range(129,159,40)]:
    fourier_data = ocean_data.v.isel(yC=0,zC=j,xC=256,time=slice(25,100))/ocean_data.v.isel(yC=0,zC=63, xC=256,time=25).max()
    transform = xf.fft(fourier_data,dim="time")
    plt.plot(transform.freq_time*2*np.pi,np.abs(transform),label=f"Level at z={ocean_data.zC[j].values}")
plt.vlines(0.5*beta*f,ymin=0,ymax=9e4,color="b",alpha=0.5,linestyles="dotted")
plt.vlines(beta*f,ymin=0,ymax=9e4,color="b",alpha=0.5,linestyles="dotted")
plt.vlines(1.5*beta*f,ymin=0,ymax=9e4,color="b",alpha=0.5,linestyles="dotted")
plt.vlines(4.5*beta*f,ymin=0,ymax=9e4,color="b",alpha=0.5,linestyles="dotted")
plt.legend(loc="upper right")
# plt.yscale("log")
# plt.gca().set_aspect(100)
plt.xlim([0,0.001])
plt.xticks(rotation=15)
# plt.xscale("symlog",linthresh=10**(-3))
plt.show()

In [ ]:
magnitude = np.abs(fourier_data)
growth_rate = sc.stats.linregress(fourier_data.time, np.log(np.abs(fourier_data/fourier_data[0])))

In [ ]:
plt.plot(time_data*fstar/(2*np.pi),magnitude)
# plt.yscale("log")

In [ ]:
growth_rate.slope

In [ ]:
normalized_fourier_data = fourier_data/np.exp(growth_rate.slope*fourier_data.time)

In [ ]:
transform

In [ ]:
# fourier_data_ni = fourier_data.drop(['intp'])
transform = xf.fft(normalized_fourier_data)
np.abs(transform).plot()
# plt.vlines(0.5*fstar,ymin=0,ymax=np.max(transform.imag)*1.1,color="b",alpha=0.5,linestyles="dotted")

In [ ]:
plt.plot(time_data,normalized_fourier_data/np.abs(normalized_fourier_data[0]))
plt.xlabel("time ($t\dfrac{f^*}{2\pi}$)")
plt.ylabel("normalized velocity ($m s^{-1}$)")
# plt.yscale("log")

In [ ]:
sr = len(time_data)/(time_data[-1]-time_data[0]).values
ts = (time_data[-1]-time_data[0])/sr
ft = np.fft.rfft(normalized_fourier_data,norm=None)
N = len(ft)
n = np.arange(N)
T = (normalized_fourier_data.time[-1]-normalized_fourier_data.time[-2]).values
freq = np.fft.rfftfreq(len(normalized_fourier_data),d=0.05)
plt.plot(freq,np.abs(ft),color="k")
# plt.vlines(0.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(1.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(2.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(3.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(4.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(5.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(6.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")

# plt.vlines(6.5*fstar*10**(4),ymin=0,ymax=0.0007)
# plt.xlim([0,0.0001])
plt.xlabel("Frequency (per intertial period)")
plt.ylabel("FFT Amplitude")

In [ ]:
# v_mean = ocean_data.v.mean(["xC"])
fourier_data_map = (ocean_data.v.isel(time=slice(200,400))/ocean_data.v.isel(time=200).max())

In [ ]:
fourier_data_map = fourier_data_map.mean("yC")
fourier_data_map

In [ ]:
transform_domain = xf.fft(fourier_data_map,dim=["zC"])

In [ ]:
plt.plot(transform_domain.freq_zC,transform_domain.isel(time=2,xC=256).real)

In [ ]:
xf.ifft(transform_domain)

In [ ]:
plt.plot(transform_domain.freq_xC,transform_domain.real[63,:,:])

In [ ]:
import matplotlib.colors as colors

In [ ]:
# plt.contourf(transform_domain.freq_xC,transform_domain.freq_time*2*np.pi,transform_domain.real[:,63,:],cmap='Accent',norm=colors.SymLogNorm(linthresh=1e9))
# plt.colorbar()
# plt.hlines(fstar,xmin=-0.1,xmax=0.1)